# Handcrafted features

In [1]:
import os
import sys 


import pandas as pd
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import warnings
import torch
from torch.utils.data import random_split

from torch.utils.data import DataLoader
import torch.nn.functional as F 
from sklearn.metrics import top_k_accuracy_score

project_path = os.path.abspath("../code")  # Adatta il percorso a dove si trova il tuo progetto
sys.path.append(project_path)
project_path = os.path.abspath("../networks")  # Adatta il percorso a dove si trova il tuo progetto
sys.path.append(project_path)
from models import *
from vipm_features import *
import vipm_costants as CONST
from vipm_pipeline import *
from dataset import *

def load_csv(csv_path):
    data = pd.read_csv(csv_path, header=None, names=['image_name', 'label'])
    return data['image_name'].tolist(), data['label'].tolist()


In [2]:
csv_path = CONST.SMALL_TRAINING_NAME_PATH
csv_unlabeled = CONST.TRAINING_NAME_PATH
csv_test = CONST.TESTING_NAME_PATH
csv_test_deg = CONST.DEG_TESTING_NAME_PATH
indir_train = CONST.SMALL_TRAINING_PATH  # Modifica in base alla posizione delle immagini
indir_test = CONST.TEST_PATH  # Modifica in base alla posizione delle immagini
indir_deg_test = CONST.DEGRADED_TEST_PATH  # Modifica in base alla posizione delle immagini
outdir = '../dataset/features'  # Modifica in base alla posizione delle feature
# Carica le immagini dal CSV
image_names, labels = load_csv(csv_path)

## KNN

### RGBMeanFeatureExtractor

In [7]:
extractor = FeatureExtractor(RGBMeanFeatureExtractor())
# Itera sugli estrattori di feature
print(f"\nTesting extractor: {extractor.name}")

try:
    X_train, y_train, _ = extractor.transform(**{"csv":csv_path, "indir":indir_train, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_train.shape[0]} feature di dimensione {X_train.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test, y_test, _ = extractor.transform(**{"csv":csv_test, "indir":indir_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test.shape[0]} feature di dimensione {X_test.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test_deg, y_test_deg, _ = extractor.transform(**{"csv":csv_test_deg, "indir":indir_deg_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test_deg.shape[0]} feature di dimensione {X_test_deg.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

models = [
    KNN(3, standardize=False),
    KNN(11, standardize=False),
    KNN(21, standardize=False),
    KNN(51, standardize=False),
    KNN(3, standardize=True),
    KNN(11, standardize=True),
    KNN(21, standardize=True),
    KNN(51, standardize=True),
]

with open('./classification_hand_crafted_rgb_knn.log', 'w') as f:

    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)
        with warnings.catch_warnings(action="ignore"):
            acc1 = top_k_accuracy_score(y_test, y_pred, k=1)
            acc5 = top_k_accuracy_score(y_test, y_pred, k=5)
            acc10 = top_k_accuracy_score(y_test, y_pred, k=10)
            y_pred = np.argmax(y_pred, axis=1)
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Model: KNN - neighbours: {model.n_neighbors} - standardize: {model.standardize}", file=f)
            print(f"Test set", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

        
        y_pred = model.predict_proba(X_test_deg)
        with warnings.catch_warnings(action="ignore"):
            acc1 = top_k_accuracy_score(y_test_deg, y_pred, k=1)
            acc5 = top_k_accuracy_score(y_test_deg, y_pred, k=5)
            acc10 = top_k_accuracy_score(y_test_deg, y_pred, k=10)
            y_pred = np.argmax(y_pred, axis=1)
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test_deg, y_pred, average='weighted')
            print(f"Test set degrated", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

            print(file=f)
            print(file=f)




Testing extractor: rgb_mean
Caricamento delle feature da ../dataset/features/train_small_rgb_mean_features_normalized.npz
Estratte 5020 feature di dimensione 3 con rgb_mean
Caricamento delle feature da ../dataset/features/test_info_rgb_mean_features_normalized.npz
Estratte 11994 feature di dimensione 3 con rgb_mean
Caricamento delle feature da ../dataset/features/test_deg_info_rgb_mean_features_normalized.npz
Estratte 11994 feature di dimensione 3 con rgb_mean


### LBPFeatureExtractor

In [8]:
extractor = FeatureExtractor(LBPFeatureExtractor())
# Itera sugli estrattori di feature
print(f"\nTesting extractor: {extractor.name}")

try:
    X_train, y_train, _ = extractor.transform(**{"csv":csv_path, "indir":indir_train, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_train.shape[0]} feature di dimensione {X_train.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test, y_test, _ = extractor.transform(**{"csv":csv_test, "indir":indir_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test.shape[0]} feature di dimensione {X_test.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test_deg, y_test_deg, _ = extractor.transform(**{"csv":csv_test_deg, "indir":indir_deg_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test_deg.shape[0]} feature di dimensione {X_test_deg.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

models = [
    KNN(3, standardize=False),
    KNN(11, standardize=False),
    KNN(21, standardize=False),
    KNN(51, standardize=False),
    KNN(3, standardize=True),
    KNN(11, standardize=True),
    KNN(21, standardize=True),
    KNN(51, standardize=True),
]


with open('./classification_hand_crafted_lbp_knn.log', 'w') as f:

    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)
        with warnings.catch_warnings(action="ignore"):
            acc1 = top_k_accuracy_score(y_test, y_pred, k=1)
            acc5 = top_k_accuracy_score(y_test, y_pred, k=5)
            acc10 = top_k_accuracy_score(y_test, y_pred, k=10)
            y_pred = np.argmax(y_pred, axis=1)
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Model: KNN - neighbours: {model.n_neighbors} - standardize: {model.standardize}", file=f)
            print(f"Test set", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

        
        y_pred = model.predict_proba(X_test_deg)
        with warnings.catch_warnings(action="ignore"):
            acc1 = top_k_accuracy_score(y_test_deg, y_pred, k=1)
            acc5 = top_k_accuracy_score(y_test_deg, y_pred, k=5)
            acc10 = top_k_accuracy_score(y_test_deg, y_pred, k=10)
            y_pred = np.argmax(y_pred, axis=1)
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test_deg, y_pred, average='weighted')
            print(f"Test set degrated", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

            print(file=f)
            print(file=f)



Testing extractor: lbp
Caricamento delle feature da ../dataset/features/train_small_lbp_features_normalized.npz
Estratte 5020 feature di dimensione 10 con lbp
Caricamento delle feature da ../dataset/features/test_info_lbp_features_normalized.npz
Estratte 11994 feature di dimensione 10 con lbp
Caricamento delle feature da ../dataset/features/test_deg_info_lbp_features_normalized.npz
Estratte 11994 feature di dimensione 10 con lbp


### LABFeatureExtractor

In [9]:
extractor = FeatureExtractor(LABFeatureExtractor())
# Itera sugli estrattori di feature
print(f"\nTesting extractor: {extractor.name}")

try:
    X_train, y_train, _ = extractor.transform(**{"csv":csv_path, "indir":indir_train, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_train.shape[0]} feature di dimensione {X_train.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test, y_test, _ = extractor.transform(**{"csv":csv_test, "indir":indir_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test.shape[0]} feature di dimensione {X_test.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test_deg, y_test_deg, _ = extractor.transform(**{"csv":csv_test_deg, "indir":indir_deg_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test_deg.shape[0]} feature di dimensione {X_test_deg.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

models = [
    KNN(3, standardize=False),
    KNN(11, standardize=False),
    KNN(21, standardize=False),
    KNN(51, standardize=False),
    KNN(3, standardize=True),
    KNN(11, standardize=True),
    KNN(21, standardize=True),
    KNN(51, standardize=True),
]

with open('./classification_hand_crafted_lab_knn.log', 'w') as f:

    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)
        with warnings.catch_warnings(action="ignore"):
            acc1 = top_k_accuracy_score(y_test, y_pred, k=1)
            acc5 = top_k_accuracy_score(y_test, y_pred, k=5)
            acc10 = top_k_accuracy_score(y_test, y_pred, k=10)
            y_pred = np.argmax(y_pred, axis=1)
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Model: KNN - neighbours: {model.n_neighbors} - standardize: {model.standardize}", file=f)
            print(f"Test set", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

        
        y_pred = model.predict_proba(X_test_deg)
        with warnings.catch_warnings(action="ignore"):
            acc1 = top_k_accuracy_score(y_test_deg, y_pred, k=1)
            acc5 = top_k_accuracy_score(y_test_deg, y_pred, k=5)
            acc10 = top_k_accuracy_score(y_test_deg, y_pred, k=10)
            y_pred = np.argmax(y_pred, axis=1)
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test_deg, y_pred, average='weighted')
            print(f"Test set degrated", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

            print(file=f)
            print(file=f)


Testing extractor: lab
Caricamento delle feature da ../dataset/features/train_small_lab_features_normalized.npz
Estratte 5020 feature di dimensione 768 con lab
Caricamento delle feature da ../dataset/features/test_info_lab_features_normalized.npz
Estratte 11994 feature di dimensione 768 con lab
Caricamento delle feature da ../dataset/features/test_deg_info_lab_features_normalized.npz
Estratte 11994 feature di dimensione 768 con lab


## Neural Networks

### RGBMeanFeatureExtractor

In [19]:
n_features = np.load(f'{outdir}/train_small_rgb_mean_features_normalized.npz')['X'].shape[1]
n_classes = 251

models = []

model = OneLayerNetwork(n_features, n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
model_option = ModelOptions(torch.nn.CrossEntropyLoss(), optimizer, scheduler, input_dim = n_features)
models.append(('OneLayerNetwork', NeuralNetwork(model, model_option)))

model = ClassifierNetwork(n_features, n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
model_option = ModelOptions(torch.nn.CrossEntropyLoss(), optimizer, scheduler, input_dim = n_features)
models.append(('ClassifierNetwork', NeuralNetwork(model, model_option)))

training_set = FeatureDataset(f"{outdir}/train_small_rgb_mean_features_normalized.npz",
                              type='test',
                              target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set = FeatureDataset(f"{outdir}/test_info_rgb_mean_features_normalized.npz",
                          type='test',
                          target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set_degraded = FeatureDataset(f'{outdir}/test_deg_info_rgb_mean_features_normalized.npz',
                                type='test',
                                target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

train_size = int(0.8 * len(training_set))  # 80% for training
val_size = len(training_set) - train_size  # Remaining 20% for validation
training_set, val_dataset = random_split(training_set, [train_size, val_size], torch.Generator().manual_seed(42))

train_loader = DataLoader(training_set, batch_size=one_layer_model_option.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=one_layer_model_option.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=one_layer_model_option.batch_size, shuffle=False)
test_degraded_loader = DataLoader(test_set_degraded, batch_size=one_layer_model_option.batch_size, shuffle=False)

with open(f'./classification_hand_crafted_rgb_nn.log', 'w') as f:
    for name, model in models:
        model.fit(train_loader, val_loader)

        mean_loss, acc1, acc5, acc10, y_pred, y_test = model.predict(test_loader)

        with warnings.catch_warnings(action="ignore"):
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Model: {name}", file=f)
            print(f"Test set", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

            
        mean_loss, acc1, acc5, acc10, y_pred, y_test = model.predict(test_degraded_loader)

        with warnings.catch_warnings(action="ignore"):
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Test set degrated", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

        print(file=f)
        print(file=f)

Epoch 1/100:
  Train Loss: 5.6461, Train Accuracy: 0.30%
  Val Loss: 5.5279, Val Accuracy: 0.30%
Epoch 2/100:
  Train Loss: 5.5738, Train Accuracy: 0.57%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 3/100:
  Train Loss: 5.5241, Train Accuracy: 0.55%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 4/100:
  Train Loss: 5.4681, Train Accuracy: 0.77%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 5/100:
  Train Loss: 5.4418, Train Accuracy: 0.85%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 6/100:
  Train Loss: 5.3989, Train Accuracy: 1.34%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 7/100:
  Train Loss: 5.3953, Train Accuracy: 1.22%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 8/100:
  Train Loss: 5.3802, Train Accuracy: 1.39%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 9/100:
  Train Loss: 5.3796, Train Accuracy: 1.17%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 10/100:
  Train Loss: 5.3688, Train Accuracy: 1.29%
  Val Loss: 5.5338, Val Accuracy: 0.20%
Epoch 11/100:
  Train Loss: 5

## LBPFeatureExtractor

In [21]:
n_features = np.load(f'{outdir}/train_small_lbp_features_normalized.npz')['X'].shape[1]
n_classes = 251

models = []

model = OneLayerNetwork(n_features, n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
model_option = ModelOptions(torch.nn.CrossEntropyLoss(), optimizer, scheduler, input_dim = n_features)
models.append(('OneLayerNetwork', NeuralNetwork(model, model_option)))

model = ClassifierNetwork(n_features, n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
model_option = ModelOptions(torch.nn.CrossEntropyLoss(), optimizer, scheduler, input_dim = n_features)
models.append(('ClassifierNetwork', NeuralNetwork(model, model_option)))

training_set = FeatureDataset(f"{outdir}/train_small_lbp_features_normalized.npz",
                              type='test',
                              target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set = FeatureDataset(f"{outdir}/test_info_lbp_features_normalized.npz",
                          type='test',
                          target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set_degraded = FeatureDataset(f'{outdir}/test_deg_info_lbp_features_normalized.npz',
                                type='test',
                                target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

train_size = int(0.8 * len(training_set))  # 80% for training
val_size = len(training_set) - train_size  # Remaining 20% for validation
training_set, val_dataset = random_split(training_set, [train_size, val_size], torch.Generator().manual_seed(42))

train_loader = DataLoader(training_set, batch_size=one_layer_model_option.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=one_layer_model_option.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=one_layer_model_option.batch_size, shuffle=False)
test_degraded_loader = DataLoader(test_set_degraded, batch_size=one_layer_model_option.batch_size, shuffle=False)

with open(f'./classification_hand_crafted_lbp_nn.log', 'w') as f:
    for name, model in models:

        model.fit(train_loader, val_loader)

        mean_loss, acc1, acc5, acc10, y_pred, y_test = model.predict(test_loader)

        with warnings.catch_warnings(action="ignore"):
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Model: {name}", file=f)
            print(f"Test set", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

            
        mean_loss, acc1, acc5, acc10, y_pred, y_test = model.predict(test_degraded_loader)

        with warnings.catch_warnings(action="ignore"):
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Test set degrated", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

        print(file=f)
        print(file=f)

Epoch 1/100:
  Train Loss: 5.6377, Train Accuracy: 0.40%
  Val Loss: 5.5250, Val Accuracy: 0.40%
Epoch 2/100:
  Train Loss: 5.5641, Train Accuracy: 0.42%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 3/100:
  Train Loss: 5.5207, Train Accuracy: 0.60%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 4/100:
  Train Loss: 5.4776, Train Accuracy: 0.60%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 5/100:
  Train Loss: 5.4518, Train Accuracy: 0.77%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 6/100:
  Train Loss: 5.4390, Train Accuracy: 0.95%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 7/100:
  Train Loss: 5.4338, Train Accuracy: 0.97%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 8/100:
  Train Loss: 5.4308, Train Accuracy: 0.85%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 9/100:
  Train Loss: 5.4282, Train Accuracy: 0.70%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 10/100:
  Train Loss: 5.4211, Train Accuracy: 1.00%
  Val Loss: 5.5302, Val Accuracy: 0.30%
Epoch 11/100:
  Train Loss: 5

## LABFeatureExtractor

In [22]:
n_features = np.load(f'{outdir}/train_small_lab_features_normalized.npz')['X'].shape[1]
n_classes = 251

models = []

model = OneLayerNetwork(n_features, n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
model_option = ModelOptions(torch.nn.CrossEntropyLoss(), optimizer, scheduler, input_dim = n_features)
models.append(('OneLayerNetwork', NeuralNetwork(model, model_option)))

model = ClassifierNetwork(n_features, n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
model_option = ModelOptions(torch.nn.CrossEntropyLoss(), optimizer, scheduler, input_dim = n_features)
models.append(('ClassifierNetwork', NeuralNetwork(model, model_option)))

training_set = FeatureDataset(f"{outdir}/train_small_lab_features_normalized.npz",
                              type='test',
                              target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set = FeatureDataset(f"{outdir}/test_info_lab_features_normalized.npz",
                          type='test',
                          target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set_degraded = FeatureDataset(f'{outdir}/test_deg_info_lab_features_normalized.npz',
                                type='test',
                                target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

train_size = int(0.8 * len(training_set))  # 80% for training
val_size = len(training_set) - train_size  # Remaining 20% for validation
training_set, val_dataset = random_split(training_set, [train_size, val_size], torch.Generator().manual_seed(42))

train_loader = DataLoader(training_set, batch_size=one_layer_model_option.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=one_layer_model_option.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=one_layer_model_option.batch_size, shuffle=False)
test_degraded_loader = DataLoader(test_set_degraded, batch_size=one_layer_model_option.batch_size, shuffle=False)

with open(f'./classification_hand_crafted_lab_nn.log', 'w') as f:
    for name, model in models:

        model.fit(train_loader, val_loader)

        mean_loss, acc1, acc5, acc10, y_pred, y_test = model.predict(test_loader)

        with warnings.catch_warnings(action="ignore"):
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Model: {name}", file=f)
            print(f"Test set", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

            
        mean_loss, acc1, acc5, acc10, y_pred, y_test = model.predict(test_degraded_loader)

        with warnings.catch_warnings(action="ignore"):
            precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
            print(f"Test set degrated", file=f)
            print(f"acc@1: {acc1} - acc@5: {acc5} - acc@10: {acc10} - prec: {precision} - rec: {recall} - fscore: {fscore}", file=f)

        print(file=f)
        print(file=f)

Epoch 1/100:
  Train Loss: 5.6667, Train Accuracy: 0.37%
  Val Loss: 5.5263, Val Accuracy: 0.50%
Epoch 2/100:
  Train Loss: 5.6115, Train Accuracy: 0.27%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 3/100:
  Train Loss: 5.5676, Train Accuracy: 0.47%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 4/100:
  Train Loss: 5.5203, Train Accuracy: 0.80%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 5/100:
  Train Loss: 5.4842, Train Accuracy: 0.85%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 6/100:
  Train Loss: 5.4539, Train Accuracy: 0.90%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 7/100:
  Train Loss: 5.4273, Train Accuracy: 1.32%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 8/100:
  Train Loss: 5.4366, Train Accuracy: 1.12%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 9/100:
  Train Loss: 5.4180, Train Accuracy: 0.97%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 10/100:
  Train Loss: 5.4209, Train Accuracy: 0.85%
  Val Loss: 5.5296, Val Accuracy: 0.20%
Epoch 11/100:
  Train Loss: 5